# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [46]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [47]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位非常聪明的生物教授。
你擅长以简洁易懂的方式回答关于生物的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

computer_template = """你是一位非常聪明的计算机教授。
你擅长以简洁易懂的方式回答关于计算机的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

literature_template = """你是一位非常聪明的汉语言文学教授。
你擅长以简洁易懂的方式回答关于汉语言文学的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

In [48]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算问题",
        "prompt_template": computer_template,
    },
        {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": literature_template,
    }
]

In [49]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [50]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [51]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [52]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [53]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [54]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算问题', '汉语言文学: 适用于回答汉语言文学问题']


In [55]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算问题
汉语言文学: 适用于回答汉语言文学问题


In [56]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [57]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算问题
汉语言文学: 适用于回答汉语言文学问题

<< INPUT >>
{input}

<<

In [58]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [59]:
print(chain.run("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '什么是黑体辐射？'}
> Finished chain.


黑体辐射是指一个完美吸收所有入射辐射的理想物体所发出的热辐射。它的颜色取决于它的温度，温度越高，发出的光的频率越高，颜色也就越接近白色。黑体辐射是研究热力学和量子力学的重要基础，也在很多实际应用中发挥着重要作用。


In [60]:
print(
    chain.run(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '找到第一个大于40的质数'}
> Finished chain.
。

首先，让我们来了解一下质数是什么。质数是指只能被1和自身整除的自然数，如2、3、5、7等。现在，我们可以开始思考如何找到第一个大于40的质数。

首先，我们可以列举出大于40的所有自然数，从最小的开始，依次为41、42、43、44...直到我们找到一个质数为止。

然后，我们可以逐个检查这些数是否为质数。我们可以用除法来检查，看看这个数能否被1和自身以外的数整除。如果不能，那么这个数就是质数。如果能够被其他数整除，那么这个数就不是质数，我们就要继续检查下一个数。

因此，在我们列举出的数中，第一个大于40的质数是41。

总结起来，我们的步骤如


In [61]:
router_chain.verbose = True

In [62]:
print(chain.run("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '什么是黑洞？'}
> Finished chain.


黑洞是一种天体，它具有非常强大的引力，甚至连光都无法逃脱。它的形成是由于某个非常大质量的恒星在死亡时发生坍缩，压缩成一个极小的空间，使得引力变得异常强大。由于黑洞具有吞噬一切的特性，所以它被称为“黑洞”。在宇宙中，黑洞是存在的，但我们无法直接观测到它们，只能通过它们对周围物质和光的影响来间接证明它们的存在。


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [63]:
print(chain.run("细胞核是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': '细胞核是什么？'}
> Finished chain.


细胞核是细胞中的一个重要部分，主要由DNA和蛋白质组成。它起着控制细胞内化学反应和遗传信息传递的作用。它类似于细胞的指挥中心，可以决定细胞的功能和特征。


In [65]:
print(chain.run("计算机网络有几层？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': '计算机网络有几层？'}
> Finished chain.


计算机网络通常被分为七层，包括物理层、数据链路层、网络层、传输层、会话层、表示层和应用层。每一层都有不同的功能和作用，通过这些层的协议和服务来实现网络通信。


In [66]:
print(chain.run("中国四大名著分别是哪四个？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语言文学: {'input': '中国四大名著分别是哪四个？'}
> Finished chain.


中国四大名著分别是《红楼梦》、《三国演义》、《水浒传》和《西游记》。
